<a href="https://colab.research.google.com/github/sebsteinig/pyGplates-PALEOMAP/blob/main/colab_pygplates_paleolocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reonstruct paleolocations
This notebook will calculate paleolocations for a given list of modern sites at a given list of ages.

In [10]:
# detect if we are running on colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  # clone the repository to access the data locally
  import os
  repo = "pyGplates-PALEOMAP"

  # remove local directory if it already exists
  if os.path.isdir(repo):
      !rm -rf {repo}

  !git clone https://github.com/sebsteinig/{repo}.git

  # install pygplates
  !sudo apt install /content/pyGplates-PALEOMAP/bin/pygplates_0.36.0_py36_ubuntu-18.04-amd64.deb

Cloning into 'pyGplates-PALEOMAP'...
remote: Enumerating objects: 685, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 685 (delta 23), reused 78 (delta 8), pack-reused 591
Receiving objects: 100% (685/685), 88.94 MiB | 13.71 MiB/s, done.
Resolving deltas: 100% (268/268), done.
Checking out files: 100% (801/801), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'pygplates' instead of '/content/pyGplates-PALEOMAP/bin/pygplates_0.36.0_py36_ubuntu-18.04-amd64.deb'
pygplates is already the newest version (0.36.0).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


## import pygplates and other packages

In [11]:
import os
import sys
# add pygplates to python part
if IN_COLAB:
  wrk_dir = repo
  sys.path.insert(0, os.path.abspath('/usr/lib')) # ubuntu VM on colab
else:
  wrk_dir = '.'
  sys.path.insert(0, os.path.abspath('./bin/pygplates_0.36.0_py37_Darwin-x86_64')) # macOS Intel 
import pygplates

import pandas as pd

## user input

In [12]:
# input csv file with label, modern latitude, modern longitude
file_input_sites = wrk_dir + '/input_sites.csv'

# list of ages (in Ma) for which we want to reconstruct paleolocations for the input sites
ages           = ['385', '380', '375', '370', '366', '359', '354', '349', '344', '339', '333', '327']

## plate model
List of available models at http://portal.gplates.org/portal/rotation_models/.
Here we are using the 'PALEOMAP PaleoAtlas for GPlates'by Scotese et al. (https://www.earthbyte.org/paleomap-paleoatlas-for-gplates/)


In [13]:
# static polygons are the 'partitioning features'
static_polygons = pygplates.FeatureCollection(wrk_dir + '/PALEOMAP_Global_Plate_Model/PALEOMAP_PlatePolygons.gpml')
# actual rotation model
rotation_model=pygplates.RotationModel(wrk_dir + '/PALEOMAP_Global_Plate_Model/PALEOMAP_PlateModel.rot')

## reconstruct points
3-step process to reconstruct paleolocations:
1. combine input points into feature collection
2. assign plate ids to features
3. reconstruct paleolocations for features

In [15]:
# load point coordinates
df_sites = pd.read_csv(file_input_sites,sep=',')

for ageCount, age in enumerate(ages):
    
    # Create target directory & all intermediate directories if don't exists
    dirName = './reconstructed-shapes/'+ ages[ageCount] + 'Ma'
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        print("Directory " , dirName ,  " Created ")
    else:    
        print("Directory " , dirName ,  " already exists") 
    
    #### step 1: put the points into a feature collection, using Lat,Lon coordinates from dataframe
    point_features = []
    for index,row in df_sites.iterrows():
        point = pygplates.PointOnSphere(float(row.LAT),float(row.LON))
        point_feature = pygplates.Feature()
        point_feature.set_geometry(point)
        point_features.append(point_feature)

    ### step 2: assign plate ids to features
    # To reconstruct any feature geometries, each feature must have a plate id assigned. If they don't already, 
    # then the pygplates function 'PlatePartitioner' performs this function (analogous to the 'assign plate ids' 
    # menu option in GPlates GUI) 
    partitioned_point_features = pygplates.partition_into_plates(static_polygons, rotation_model, point_features) 

    ### step 3: reconstruct paleolocations for features
    # Two possible ways:
    
    # 1. save shape files to disk for later use (e.g. load sahpefiles into python script for direct plotting)
    pygplates.reconstruct(partitioned_point_features, rotation_model, dirName +'/NA-sites.shp', float(ages[ageCount]))
    
    # 2. output paleolocations directly
    reconstructed_feature_geometries = []
    pygplates.reconstruct(partitioned_point_features, rotation_model, reconstructed_feature_geometries, float(ages[ageCount]))    
    for siteCount, reconstructed_feature_geometry in enumerate(reconstructed_feature_geometries):
        paleoLocation = reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon()
        print('Paleolocation for ' + df_sites.name[siteCount] + ' at ' + ages[ageCount] + 'Ma (LAT/LON): ' + str(round(paleoLocation[0],1)) + '/'+str(round(paleoLocation[1],1)) )


Directory  ./reconstructed-shapes/385Ma  already exists
Paleolocation for Shaffer-Rohling_Core at 385Ma (LAT/LON): -35.7/-46.7
Paleolocation for I35_Sycamore_South at 385Ma (LAT/LON): -37.4/-47.0
Paleolocation for Kansas_OK_Outcrop at 385Ma (LAT/LON): -37.1/-43.7
Paleolocation for Hamsten_Unit_Core at 385Ma (LAT/LON): -34.6/-46.4
Paleolocation for Bakken at 385Ma (LAT/LON): -18.7/-42.6
Directory  ./reconstructed-shapes/380Ma  already exists
Paleolocation for Shaffer-Rohling_Core at 380Ma (LAT/LON): -33.9/-46.5
Paleolocation for I35_Sycamore_South at 380Ma (LAT/LON): -35.7/-46.8
Paleolocation for Kansas_OK_Outcrop at 380Ma (LAT/LON): -35.4/-43.6
Paleolocation for Hamsten_Unit_Core at 380Ma (LAT/LON): -32.9/-46.3
Paleolocation for Bakken at 380Ma (LAT/LON): -17.0/-42.6
Directory  ./reconstructed-shapes/375Ma  already exists
Paleolocation for Shaffer-Rohling_Core at 375Ma (LAT/LON): -32.2/-46.4
Paleolocation for I35_Sycamore_South at 375Ma (LAT/LON): -34.0/-46.6
Paleolocation for Kansas_O